## Integrating Python Code into Non-Python codebases  
    aka., How to inflict Python on your friends and enemies.
    aka., Hey supposedly this problem doesn't exist in .NET
Adam Shaver, adam.shaver@gmail.com, July 15th, 2015

#### Overview and Summary
This notebook covers three ways to integrate Python with C/C++ and a final method that allows integration to other code via a Service Oriented Architechture. The C/C++ integration section covers C-Types, PyObject, and Boost Python, whereas the SOA introduces Zato.io. Each section is orthogonal and is meant to stand alone.


#### ToolChain for the notebook
- Clean Ubuntu 12.04 Desktop LTS (GCC 4.8.2, Python 2.7.6)
  - sudo apt-get install -y build-essential python-dev libbz2-dev
- Boost 1.58.0
 - Compiled from source, just good practice to make sure everything works on the system. As root
   - mkdir -p /usr/local/boost/1.58
   - boostrap.sh –prefix=/usr/local/boost/1.58
   - b2.sh -j<n> install
- Anaconda
- WinPDB 1.4.8 
 - conda install wxpython
 - python setup.py install -f
- Zato.io (see the topic for install info)
 
#### Motivation
- Using Python for prototyping
- Using Python as middleware/glue between legacy code
- There are things that each language does well (C++, Fortran, Python) 
- Code Obfuscation
- Unit tests framework in one language, code in another
- If you're here, you probably have a reason

#### Avenues of Integration
- Topic 1: Ctypes : Python --> C/C++ 
- Topic 2: pyObject : Python <--> C/C++
- Topic 3: The Boost Python C++ Library: C/C++ <--> Python
- Topic 4: ESB (Zato.io) Python <--> [Anything]
- Addendum: Checking for a leaky C/C++ interface with Valgrind

*Note, there are plenty more.* Also, it's assumed if you employ item 4 from above, you can do pretty much anything.

In [1]:
from __future__ import print_function

## Topic 1: Ctypes, Python --> C/C++, using simple types from the C Language
Ctypes are defined mappings between C and Python.
- https://docs.python.org/2/library/ctypes.html
- https://docs.python.org/3/library/ctypes.html
- https://wakari.io/nb/url///wakari.io/static/notebooks/Lecture_6A_Fortran_and_C.ipynb


They serve to allow passing data to/from C/C++ functions that expect C-like data. Note, if the library expects some more complicated data structure/class you might be out of luck.

In [2]:
import ctypes
curl_loc = ctypes.util.find_library('curl')
print(curl_loc)

libcurl.so.4


In [3]:
!locate libcurl

/home/ashaver/anaconda/lib/libcurl.a
/home/ashaver/anaconda/lib/libcurl.la
/home/ashaver/anaconda/lib/libcurl.so
/home/ashaver/anaconda/lib/libcurl.so.4
/home/ashaver/anaconda/lib/libcurl.so.4.3.0
/home/ashaver/anaconda/lib/pkgconfig/libcurl.pc
/home/ashaver/anaconda/pkgs/curl-7.38.0-0/lib/libcurl.a
/home/ashaver/anaconda/pkgs/curl-7.38.0-0/lib/libcurl.la
/home/ashaver/anaconda/pkgs/curl-7.38.0-0/lib/libcurl.so
/home/ashaver/anaconda/pkgs/curl-7.38.0-0/lib/libcurl.so.4
/home/ashaver/anaconda/pkgs/curl-7.38.0-0/lib/libcurl.so.4.3.0
/home/ashaver/anaconda/pkgs/curl-7.38.0-0/lib/pkgconfig/libcurl.pc
/usr/lib/x86_64-linux-gnu/libcurl-gnutls.so.3
/usr/lib/x86_64-linux-gnu/libcurl-gnutls.so.4
/usr/lib/x86_64-linux-gnu/libcurl-gnutls.so.4.3.0
/usr/lib/x86_64-linux-gnu/libcurl.so.3
/usr/lib/x86_64-linux-gnu/libcurl.so.4
/usr/lib/x86_64-linux-gnu/libcurl.so.4.3.0
/usr/share/doc/libcurl3
/usr/share/doc/libcurl3-gnutls
/usr/share/doc/libcurl3/NEWS.Debian.gz
/usr/share/doc/libcurl3/changelog.Debia

Examine all the objects. Then the exported objects. Note, if you write C++ code you wil need to prevent name mangling with the directive ``` extern "C" ``` 

In [4]:
!nm -D /usr/lib/x86_64-linux-gnu/libcurl.so.4 

                 U accept
                 U access
                 U ASN1_INTEGER_get
                 U ASN1_STRING_data
                 U ASN1_STRING_length
                 U ASN1_STRING_print
                 U ASN1_STRING_to_UTF8
                 U ASN1_STRING_type
                 U ber_free
                 U ber_memfree
                 U ber_sockbuf_add_io
                 U bind
                 U BIO_ctrl
                 U BIO_free
                 U BIO_new
                 U BIO_s_mem
                 U BN_bn2bin
                 U BN_num_bits
                 U calloc
                 U clock_gettime
                 U close
                 U connect
                 U CRYPTO_cleanup_all_ex_data
                 U CRYPTO_free
                 U CRYPTO_malloc
                 U __ctype_b_loc
                 U __ctype_tolower_loc
000000000002a860 T curl_easy_cleanup
000000000002a900 T curl_easy_duphandle
0000000000023ab0 T curl_easy_escape
000000000002a8c0 T curl_easy

In [5]:
!nm -D /usr/lib/x86_64-linux-gnu/libcurl.so.4 |grep "T "

000000000002a860 T curl_easy_cleanup
000000000002a900 T curl_easy_duphandle
0000000000023ab0 T curl_easy_escape
000000000002a8c0 T curl_easy_getinfo
000000000002a550 T curl_easy_init
000000000002acc0 T curl_easy_pause
000000000002a630 T curl_easy_perform
000000000002ae30 T curl_easy_recv
000000000002aaf0 T curl_easy_reset
000000000002aea0 T curl_easy_send
000000000002a590 T curl_easy_setopt
00000000000359c0 T curl_easy_strerror
0000000000023da0 T curl_easy_unescape
0000000000023c00 T curl_escape
000000000000c710 T curl_formadd
000000000000d4e0 T curl_formfree
000000000000db40 T curl_formget
0000000000023e00 T curl_free
00000000000362c0 T curl_getdate
0000000000023a70 T curl_getenv
000000000002a4f0 T curl_global_cleanup
000000000002a370 T curl_global_init
000000000002a430 T curl_global_init_mem
00000000000253d0 T curl_maprintf
0000000000025740 T curl_mfprintf
0000000000025690 T curl_mprintf
0000000000023f80 T curl_msnprintf
00000000000255e0 T curl_msprintf
0000000000031bf0 T curl_multi_

---
We can load libraries in 4 ways using ctypes (CDLL, OleDLL, WinDLL, PyDLL). Items 2,3 are Windows only. You can use this method if the function signature is well defined, documented (e.g., can be formed with ctypes).

In [6]:
_curl = ctypes.CDLL(curl_loc)
_curl_version_info = _curl.curl_version_info
print(_curl_version_info())
easy_handle = _curl.curl_easy_init()

# This magic number came from a header typdef enum
_curl.curl_easy_setopt(easy_handle, 0x100000+1 , "http://example.org") 

1241254656


48

In [14]:
%%file code/hello.cpp
#include <iostream>
#include <string.h>
using namespace std;
extern "C" {
    int main () {
        cout << "Hello Main2\n";
        return 42;
    }
    
    string helloStr() {
        return "Hello Str\n";
    }
    
    const char* helloStrPtr() {
        char* p = new char[19];
        strcpy(p,"Hello Str Pointer\n");
        return p;
    }
    
    void helloStrRef(char* p) {
        strcpy(p,"Hello Str Ref!\n");
    }
    
    struct MsgWrapper {
        char * msgIn;
        char * msgOut;
    };
    
    void helloStrRefFromStruct(MsgWrapper s) {
        strcpy(s.msgOut, "Aloha\n");
    }
    
    int missing_answer(int  i) {
        i = (42-i);
        return i;
    }
    
     
}

Overwriting code/hello.cpp


Compile to a shared library. Note, if you are on Windows you'll need to declare an export symbol or use an export library. In Linux, because all dynamic libraries share a heap, all the symbols are exported. 

In [15]:
! rm bin/hello bin/libHello.so
!g++ -c -std=c++11 -fPIC code/hello.cpp -o code/hello.o
!g++ -o bin/libHello.so -shared code/hello.o
!g++ -o bin/hello code/hello.o
!bin/hello

Hello Main2


In [13]:
!nm -D bin/libHello.so |grep 'T '

0000000000000d28 T _fini
00000000000009d8 T _init
0000000000000b95 T main
0000000000000c19 T _Z11helloStrPtrv
0000000000000c5e T _Z11helloStrRefPc
0000000000000cb2 T _Z14missing_answeri
0000000000000c87 T _Z21helloStrRefFromStruct10MsgWrapper
0000000000000bb6 T _Z8helloStrv


###Executing a function. 
(1) As an aside, I routinely import ctypes in each cell and reload the library, because if anything goes wrong, the kernel crashes (and the dynamic loaded library is lost). (2) Also, I explicitly didn't import * from cTypes so you can see which functions are from the module.

In [19]:
import ctypes
_hello = ctypes.CDLL('./bin/libHello.so')
print(_hello.main()) # Note this prints the return code, not hello
print(_hello.helloStr()) # Note this doesn't wactually pass back a string, but the pointer to a string.




42
1347556240


#### Relevant nugget about ctypes
 - Python coding convention says that libraries in C/C++ that mask the same library in python should be brought into the namespace as _foo, instead of foo
 - There is no good way to remove libraries from the namespace (e.g., del) once they've been loaded by ctypes.CDLL

Executing a function and catching the result. Note, it is pointer to a string, so we will have to dereference it to get the string value. 

In [20]:
import ctypes
_hello = ctypes.CDLL('./bin/libHello.so')
a_str_ptr = _hello.helloStrPtr() # Returns a pointer to a string (char* or c_char_p)
print(a_str_ptr)
#print(a_str_ptr.value) Fails to deref as python doesn't have it cast as a cType.c_char_p.
print(ctypes.c_char_p(a_str_ptr).value) # Dereferences it to get the char

36753856
Hello Str Pointer



We can also pass a reference to a string into the code.

In [21]:
import ctypes
_hello = ctypes.CDLL('./bin/libHello.so')
a_str_ptr2 = ctypes.create_string_buffer('\000' * 45)
_hello.helloStrRef(ctypes.byref(a_str_ptr2)) # passes in the pointer 
print(a_str_ptr2)
print(a_str_ptr2.value) 

Hello Str Ref!



We can even generate structs and such to pass the data back and forth

In [22]:
import ctypes
_hello = ctypes.CDLL('./bin/libHello.so')
class MsgWrapper(ctypes.Structure):
    _fields_ = [("msgOut", ctypes.c_char_p), ("msgIn", ctypes.c_char_p)]
amsg = MsgWrapper("Goodbye", "Hello")
print(amsg.msgIn)
print(amsg.msgOut)


Hello
Goodbye


In [23]:
_hello.helloStrRefFromStruct(amsg)
print(amsg.msgIn)
print(amsg.msgOut)

Aloha

Goodbye


In [24]:
import ctypes
_helloCpp = ctypes.CDLL('./bin/libHello.so')
print(_helloCpp.missing_answer(24))

18


###Pitfalls with ctypes
----
 - Endian specification
 - C++ name mangling (fixed with Extern C)
 - memory leaks (allocation in c/c++ without deallocation). These can easily happen across the interface if you're not passing by reference (with byref).
 - pointer problems (out of scope, dereference, etc). This will generally just crash the kernel.

What about more complicated input and output? C-types only have structs and unions, but not classes. Therefore, we can't pass around objects from a C++ library. Enter pyObject......

---
##Topic 2: pyObject,  Python in modules C/C++ 

A pyObject is a Python object in C.  Also, *everything* in Python is really a pyObject. Using it, we get inerfaces that can do anything python can do (though the wrapping of complicated Python objects requires equally complicated code). Strangely, everything I've seen uses pyObjects to make modules, rather than some lightweight interface like ctypes. 
 - https://docs.python.org/2/extending/extending.html
 - https://docs.python.org/3/extending/extending.html
 - http://intermediate-and-advanced-software-carpentry.readthedocs.org/en/latest/c++-wrapping.html
 - (my favorite, includes numpy headers, deploying with setup.py, etc) http://dan.iel.fm/posts/python-c-extensions/
 
To start, what you need in a module in c++
  1. Function: The function which takes and possibly returns a pyObject
  1. Method Table: A static definition of the function signature, how it is called, it's name in python, in C/C++
  1. Initialization: An initialization call for the function (which I belive registers the module and this function as a function of the module.

In [25]:
%%file code/Simple.cpp
#include <python2.7/Python.h>  //Contains the spec for the Python API, pyObject, etc.

//Function 1
static PyObject * answer_to_everything(PyObject *self, PyObject *args)
{
    return PyLong_FromLong(42); // convert to Python object of a type
}

//Function 2
static PyObject * missing_from_answer_to_everything(PyObject *self, PyObject *args)
{
    int sts;
    if (!PyArg_ParseTuple(args, "i", &sts)) //"((ii)(ii))(ii)", is f((0, 0), (400, 300)), (10, 10))
        return NULL;                        //"s|si", is f('wer','d', 10) or f('cmd')
    sts = 42 - sts;
    PyObject * ret = PyLong_FromLong(sts); // convert to Python object of a type
    return ret;
}

// Static that maps C names to python and outlines the variables
static PyMethodDef SimpleMethods[] = {
    {"what_is_the_answer",  answer_to_everything, METH_VARARGS,
     "Gets The Answer."}, 
    {"missing_answer",  missing_from_answer_to_everything, METH_VARARGS,
     "Finds what you're missing."},
    {NULL, NULL, 0, NULL}        /* Sentinel */ //Apparently required for table termination....
};



// Registration of the functions and generation of the module
PyMODINIT_FUNC
initSimple(void)  // MUST be named "initFoo" for a library with name of "Foo"
{
    (void) Py_InitModule("Simple", SimpleMethods);
}

Writing code/Simple.cpp


When compiling this, you have to explicitly link in the python library. Otherwise, the objects will compile, but the functions for things like (PyArg_ParseTuple won't be defined).
Also, note that the PyMODINIT_FUNC makes the init an "extern C" function without mangling.

In [26]:
! rm bin/Simple.so Simple.so
!g++ -c -std=c++11 -fPIC code/Simple.cpp -o code/Simple.o 
!g++ -o bin/Simple.so -shared code/Simple.o -L/usr/lib -lpython2.7
!nm -D bin/Simple.so |grep 'T '
!ln -s ./bin/Simple.so Simple.so

rm: cannot remove ‘bin/Simple.so’: No such file or directory
rm: cannot remove ‘Simple.so’: No such file or directory
000000000000086c T _fini
0000000000000658 T _init
0000000000000843 T initSimple


In [30]:
import Simple as _Simple
print(_Simple.what_is_the_answer(234234))
print(_Simple.missing_answer(19))


42
23


### Passing Functions into C++ from Python
The PyObject framework allows one to pass any Python object back and forth. Because everything in Python is an object, we can pretty much do anything. Here we pass in a simple function, and execute it (see simple function). When someone does this, typically it's to modify the Python data with C++ (see pedantic function).

In [33]:
%%file code/SimpleFcn.cpp
#include <python2.7/Python.h>  //Contains the spec for the Python API, pyObject, etc.

// First we need to accept and set Python function as the callback
// Note, this set function was taken right from the Python Documentation
static PyObject *my_callback = NULL;
static PyObject * my_set_callback(PyObject *dummy, PyObject *args)
{
    PyObject *result = NULL;
    PyObject *temp;

    if (PyArg_ParseTuple(args, "O:set_callback", &temp)) {
        if (!PyCallable_Check(temp)) {
            PyErr_SetString(PyExc_TypeError, "parameter must be callable");
            return NULL;
        }
        Py_XINCREF(temp);         /* Add a reference to new callback */
        Py_XDECREF(my_callback);  /* Dispose of previous callback */
        my_callback = temp;       /* Remember new callback */
        /* Boilerplate to return "None" */
        Py_INCREF(Py_None);
        result = Py_None;
    }
    return result;
}


//Now we call the funciton
static PyObject * simple_callback(PyObject *self, PyObject *args)
{
    return PyObject_CallObject(my_callback, args);
}

//Calling a more pedantic fashion to highlight decrementing for garbage collection
//Also, we could do something with the result in C/C++ if we wanted
//Note, modified from  Python Documetation
static PyObject * pedantic_callback(PyObject *self, PyObject *args)
{
    double val_in;
    if (!PyArg_ParseTuple(args, "d", &val_in)) 
        return NULL;  
    val_in = val_in*42;
    
    PyObject *arglist;
    PyObject *result;
    arglist = Py_BuildValue("(d)", val_in); //Note "tuplization"
    //return arglist; //Actually breaks from Python and returns (foo,) rather than (foo)
    result = PyObject_CallObject(my_callback, arglist);
    Py_DECREF(arglist); // Note decref to release args. It was only needed for the fcn call.
    //return arglist; // Would return ((<NULL>,),)
    return result;
    
}

// Static that maps C names to python and outlines the variables
static PyMethodDef SimpleMethods[] = {
    {"set_fcn_callback",  my_set_callback, METH_VARARGS,
     "Sets_a_callback."},
    {"simple_callback",  simple_callback, METH_VARARGS,
     "calls back with passthrough args."},
    {"pedantic_callback",  pedantic_callback, METH_VARARGS,
     "calls back with explicit int args."},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

// Registration of the functions and generation of the module
PyMODINIT_FUNC
initSimpleFcn(void)  // MUST be named "initFoo" for a library with name of "Foo"
{
    (void) Py_InitModule("SimpleFcn", SimpleMethods);
}

Writing code/SimpleFcn.cpp


We can make this in to an installable module without much effort.

In [31]:
%%file code/setup.py

from distutils.core import setup, Extension

setup(
    ext_modules=[Extension("SimpleFcn", ["SimpleFcn.cpp"])],
)


Writing code/setup.py


Go build this in conda's python via ```python setup.py build_ext --inplace```

In [34]:
! rm bin/SimpleFcn.so SimpleFcn.so
!g++ -c -std=c++11 -fPIC code/SimpleFcn.cpp -o code/SimpleFcn.o 
!g++ -o bin/SimpleFcn.so -shared code/SimpleFcn.o -L/usr/lib -lpython2.7
!nm -D bin/SimpleFcn.so |grep 'T '
!ln -s ./bin/SimpleFcn.so SimpleFcn.so

rm: cannot remove ‘bin/SimpleFcn.so’: No such file or directory
0000000000000bb0 T _fini
00000000000007e8 T _init
0000000000000b85 T initSimpleFcn


We can run this function. It also allows us a chance to look at the call stack from C++ up through the notebook

In [1]:
import SimpleFcn
del SimpleFcn
import SimpleFcn
def what_is_the_answer_with_inflation(rate):
    #%%debug
    #import rpdb2; rpdb2.start_embedded_debugger('12345')
    return 42*rate
print(what_is_the_answer_with_inflation(1.02))   
SimpleFcn.set_fcn_callback(what_is_the_answer_with_inflation)
print(SimpleFcn.simple_callback(1.02))

# Python instances 42 joined strings of "1.07"
print(SimpleFcn.simple_callback("1.07"))

42.84
42.84
1.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.071.07


In [5]:
print(SimpleFcn.pedantic_callback(1.02))

# Will break because we're trying to extract a float
print(SimpleFcn.pedantic_callback(1.07))

1799.28
1887.48


###Pitfalls with pyObject
 - The *first* line of the C/C++ file should be '''#include < python2.7/Python.h\> ''' because it may alter how other headers are loaded (such as iostream). 
   - Note, most tutorials will have you just write '''< Python.h\>''', which will not find the file (at least for Unbuntu)
   - The Python headers are available in Ubuntu via python-dev  
   - When linking, you need to include the python library libpython<ver>. In this case, it's libpython2.7.so.
 - Reference counting needs to be done correctly to prevent memory leaks when variables are generated on one side and destroyed on another. If the same variable is passed back/forth as a carrier, then the possibility of leads is reduced (see the struct message example).
 - You can run into strange errors when you're not converting to the right Python type (e.g., long when int needed, etc).
   - This is all documented in the c-api documentation.

---
##Topic 3: Boost Python,  "Seamless" Python and Boost Integration

First, let's examine the example from boost (boost_1_58_0/libs/python/doc/index.html).
 - Note, in order for this to run, I needed to put these libraries on the path before starting the notebook: "export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/boost/1.58/lib
"
 - If you notice, this the easiest implementation so far (nearly everything is hidden by BOOST_PYTHON_MODULE)

In [ ]:
# Will allow compiling, but will still fail at runtime.
import os
print os.environ['LD_LIBRARY_PATH']
os.environ['LD_LIBRARY_PATH']='/usr/local/boost/1.58/lib'

In [6]:
%%file code/hello_ext.cpp
#include <boost/python.hpp>

const char * greet()
{
   return "hello, world";
}

BOOST_PYTHON_MODULE(hello_ext) //Will define the module init name via macros (see objects below)
{
    using namespace boost::python;
    def("greet", greet);
}


Writing code/hello_ext.cpp


In [7]:
!rm bin/hello_ext.so hello_ext.so
!g++ -c -std=c++11 -fPIC code/hello_ext.cpp -o code/hello_ext.o -I/home/ashaver/anaconda/include/python2.7 -I/usr/local/boost/1.58/include
!g++ -o bin/hello_ext.so -shared code/hello_ext.o /usr/local/boost/1.58/lib/libboost_python.so
!nm -D bin/hello_ext.so |grep 'T '
!ln -s ./bin/hello_ext.so hello_ext.so
!ldd hello_ext.so


rm: cannot remove ‘bin/hello_ext.so’: No such file or directory
rm: cannot remove ‘hello_ext.so’: No such file or directory
0000000000005dc8 T _fini
0000000000004948 T _init
0000000000005002 T inithello_ext
000000000000501e T _Z21init_module_hello_extv
0000000000004ff5 T _Z5greetv
	linux-vdso.so.1 =>  (0x00007fff70ffc000)
	libboost_python.so.1.58.0 => /usr/local/boost/1.58/lib/libboost_python.so.1.58.0 (0x00007f31241cb000)
	libstdc++.so.6 => /usr/lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007f3123eaf000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007f3123c98000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f31238d2000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f31235cc000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f3124623000)


In [8]:
# Must have python on the path.
import hello_ext
print hello_ext.greet()

hello, world


In [9]:
%%file code/boost42.cpp
#include <boost/python.hpp>
        
int missing_answer(int a) {
    return (42-a);
}

BOOST_PYTHON_MODULE(boost42) //Will define the module init name via macros (see objects below)
{
    using namespace boost::python;
    def("missing_answer", missing_answer);
}



Writing code/boost42.cpp


In [10]:
!rm bin/boost42.so boost42.so
!g++ -c -std=c++11 -fPIC code/boost42.cpp -o code/boost42.o -I/home/ashaver/anaconda/include/python2.7 -I/usr/local/boost/1.58/include
!g++ -o bin/boost42.so -shared code/boost42.o /usr/local/boost/1.58/lib/libboost_python.so
!nm -D bin/boost42.so |grep 'T '
!ln -s ./bin/boost42.so boost42.so
!ldd boost42.so

rm: cannot remove ‘bin/boost42.so’: No such file or directory
rm: cannot remove ‘boost42.so’: No such file or directory
0000000000006c20 T _fini
00000000000053b0 T _init
0000000000005b76 T initboost42
0000000000005b65 T _Z14missing_answeri
0000000000005b92 T _Z19init_module_boost42v
	linux-vdso.so.1 =>  (0x00007fff87ffc000)
	libboost_python.so.1.58.0 => /usr/local/boost/1.58/lib/libboost_python.so.1.58.0 (0x00007f555a4f0000)
	libstdc++.so.6 => /usr/lib/x86_64-linux-gnu/libstdc++.so.6 (0x00007f555a1d4000)
	libgcc_s.so.1 => /lib/x86_64-linux-gnu/libgcc_s.so.1 (0x00007f5559fbd000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f5559bf7000)
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007f55598f1000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f555a94a000)


In [11]:
# Must have python on the path.
import boost42
print boost42.missing_answer(23)

19


We can call python from boost in various ways. You can pass in fuctions just like with C++. Additionally, you can have c++ call/evaluate python directly. If you're reading the Boost documentation they call this "embedding". 

 - object eval(str expression, object globals = object(), object locals = object())
 - object exec(str code, object globals = object(), object locals = object())
 - object exec_file(str filename, object globals = object(), object locals = object())
 - or...import a module and work within its namespace

In [12]:
%%file code/boostUsingPython.cpp
#include <boost/python.hpp>
#include <iostream>
using namespace boost::python;
        
int main() {
    Py_Initialize();
    object random_module = import("random");
    object rand_namespace = random_module.attr("__dict__");
    object ignored = exec("result = gauss(2,5)", rand_namespace);
    double val = extract<double>(rand_namespace["result"]);
    std::cout << val <<'\n';
    return 0;
};

Writing code/boostUsingPython.cpp


In [13]:
!g++ -c -std=c++11 -fPIC code/boostUsingPython.cpp -o code/boostUsingPython.o -I/home/ashaver/anaconda/include/python2.7 -I/usr/local/boost/1.58/include
!g++ -o bin/boostUsingPython code/boostUsingPython.o /usr/local/boost/1.58/lib/libboost_python.so -lpython2.7
!bin/boostUsingPython
!bin/boostUsingPython
!bin/boostUsingPython
!bin/boostUsingPython
!bin/boostUsingPython


5.56215
9.46684
5.11751
1.28348
0.42816


###Pitfalls with Boost Python
----
 - Compiling boost, not having libraries on the path at runtime (which is easy to catch)
 - Not extracting the correct type
 - If something goes wrong in the macros, you would have to read the C-header compiled code (before the object file is made)
 - Not putting the #include < boost/python.hpp > at the top as it drives other headers
 - Forgetting to link in the boost python library 
 - Also memory leaks (though boost employs smart pointers to do reference counting of the everything generated via Boost). Also, you can smart-pointer wrap any new() declarations with Boost's smart pointers, if you want.

In [14]:
import ctypes
helloCpp = ctypes.CDLL('./bin/libHello.so')
print(helloCpp.missing_answer(24)) 
%timeit helloCpp.missing_answer(24)
%time helloCpp.missing_answer(24)

import Simple
print(Simple.missing_answer(24))
%timeit Simple.missing_answer(24)
%time Simple.missing_answer(24)

import boost42
print(boost42.missing_answer(24))
%timeit boost42.missing_answer(24)
%time boost42.missing_answer(24)

18
The slowest run took 29.85 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 272 ns per loop
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15 µs
18
The slowest run took 23.83 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 170 ns per loop
CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.1 µs
18
The slowest run took 23.18 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 216 ns per loop
CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 13.8 µs


18

---
##Topic 4: Zato.io, using an ESB to integrate Python

Installation/download is different for each platform:
- https://zato.io/downloads.html
- https://zato.io/downloads-2.0-ubuntu.html
- http://www.linuxjournal.com/content/zato%E2%80%94agile-esb-soa-rest-and-cloud-integrations-python
- https://zato.io/docs/2.0/index.html

My install instrutions (curl didn't work, substituted wget and cat). 

---

- sudo apt-get install apt-transport-https
- sudo apt-get install python-software-properties software-properties-common
- wget https://zato.io/repo/zato-0CBD7F72.pgp.asc
- cat zato-0CBD7F72.pgp.asc | sudo apt-key add -
- sudo apt-add-repository https://zato.io/repo/stable/2.0/ubuntu
- sudo apt-get update
- sudo apt-get -y install zato
- reboot, as the install made a user 'zato ESB'
---
  

####Tutorial (From Zato)
 - Option 1, using Docker (https://zato.io/docs/2.0/admin/guide/install-docker.html)
 - Option 2
  - Getting redis (http://redis.io/)
    - wget http://download.redis.io/releases/redis-3.0.2.tar.gz
    - tar -xzvf redis-3.0.2.tar.gz
    - cd redis-3.0.2
    - make 
    - make test
    - sudo make install
    - cd utils
    - sudo ./install_server.sh
   
    ```
    Success!
    Starting Redis server...
    Installation successful! 
    ```
  - cd /usr/local/bin
  - sudo ln -s /opt/zato/2.0.5/bin/zato zato # Was missing line in the tutorial...zato wasn't in bin.
  - sudo su zato 
  - cd ~
  - mkdir -p env/qs-1
  - /opt/zato/2.0.5/bin/zato quickstart create ~/env/qs-1 sqlite localhost 6379 --kvdb_password '' --verbose
  - This output is **important** as it contains log-in info for administration!
 ```
 [1/8] Certificate authority created
[2/8] ODB schema created
[3/8] ODB initial data created
[4/8] server1 created
[5/8] server2 created
[6/8] Load-balancer created
Superuser created successfully.
[7/8] Web admin created
[8/8] Management scripts created
Quickstart cluster quickstart-150282 created
Web admin user:[admin], password:[suri-rona-esar-eruk]
Start the cluster by issuing the /opt/zato/env/qs-1/zato-qs-start.sh command
Visit https://zato.io/support for more information and support options
```
  - /opt/zato/env/qs-1/zato-qs-start.sh
  ```
Starting the Zato cluster quickstart-578017
Running sanity checks
[1/6] Redis connection OK
[2/6] SQL ODB connection OK
[3/6] Load-balancer started
[4/6] server1 started
[5/6] server2 started
[6/6] Web admin started
Zato cluster quickstart-578017 started
Visit https://zato.io/support for more information and support options
```
  - zato check-config /opt/zato/env/qs-1/server1
  - zato check-config /opt/zato/env/qs-1/server2
  - curl localhost:11223/zato/ping ; echo
  

In [15]:
%%file zato_tutorial/my_service.py

from zato.server.service import Service

class GetClientDetails(Service):
    def handle(self): # The only method that *has* to be defined. This is the event-triggered entry point.
        self.log_input()

Overwriting zato_tutorial/my_service.py


In [17]:
!curl localhost:11223/tutorial/first-service -d '{"cust_id":123, "cust_type":"A"}'
    

Go read and work through https://zato.io/docs/tutorial/02.html

IMHO, worth it for
  - explanation of outgoing messages and external servers. (They do a CRM example).
  - introduction to using Simple IO to insert a lightweight interface layer between differing fields of incoming/outgoing messages
  - an introduction to zmq, a really lightweight messaging protocol. 
  - gunicorn (a python port of ruby's unicorn for the reactor web-pattern "green unicorn")

For now, we'll look at other modifications to Zato through packages.:

In [18]:
!ls /opt/zato/2.0.5/bin/

activate	  glance	   nova			  pyrsa-keygen
activate.csh	  gunicorn	   ntlm_example_extended  pyrsa-priv2pub
activate.fish	  gunicorn_django  ntlm_example_simple	  pyrsa-sign
activate_this.py  gunicorn_paster  pep8			  pyrsa-verify
alembic		  ipython	   pip			  python
behave		  jsondiff	   pip2			  python2
buildout	  jsonpatch	   pip2.7		  python2.7
coverage	  jsonpointer	   py			  raven
coverage2	  keyczart	   pybabel		  retools-worker
coverage-2.7	  keyring	   pyflakes		  rpdb2
cygdb		  keystone	   pygmentize		  watchmedo
cython		  mllp_send	   pyprof2calltree	  winpdb
easy_install	  ndg_httpclient   pyrsa-decrypt	  zato
easy_install-2.7  newrelic-admin   pyrsa-decrypt-bigfile
f2py		  nosetests	   pyrsa-encrypt
flake8		  nosetests-2.7    pyrsa-encrypt-bigfile


We're going to install our package Simple from earlier. 
 - cp simplePkg .
 - /opt/zato/2.0.5/bin/python setup.py build_ext install
 
This is similar to installing packages into the environment for Anaconda. Note, here we've used something from Topic 2 (PyObjects) inside the ESB.

In [19]:
%%file zato_tutorial/my_answer.py

from zato.server.service import Service
import Simple # original line

class GetClientAnswer(Service):
    def handle(self): # The only method that *has* to be defined. This is the event-triggered entry point.
        self.logger.info('Request Answer: {}'.format(self.request.payload))

       
        response = {}
        q = self.request.payload['question']
        response['question'] = q
        response['answer'] = Simple.missing_answer(q) # Original Line

        self.logger.info('Response: {}'.format(response))
       
        
        # And return response to the caller
        self.response.payload = response

Overwriting zato_tutorial/my_answer.py


Copy to the hot-load. Then add the channel for the service. Check the logs.

In [21]:
!curl localhost:11223/talk/answer -d '{"question":23}'
!curl localhost:11223/talk/answer -d '{"question":21}'
!curl localhost:11223/talk/answer -d '{"question":22342341}'


{"answer": 19, "question": 23}{"answer": 21, "question": 21}{"answer": -22342299, "question": 22342341}

In [22]:
import Simple
%time Simple.missing_answer(24)
%time !curl localhost:11223/talk/answer -d '{"question":23}'

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 22.9 µs
{"answer": 19, "question": 23}CPU times: user 58 µs, sys: 4.07 ms, total: 4.13 ms
Wall time: 118 ms


# Let's  address something that might happen in the real world...
The company buys an expensive peice of software. They buy only one copy. You would like to use python to allow everyone to use it. In this specific case, it's a Linear-Programming Solver, Coin (which is free, so your company probably paid too much).

Go install CyLP (the python front end for several COIN-OR packages)
 - http://mpy.github.io/CyLPdoc/
 - https://projects.coin-or.org/CyLP
 - https://pypi.python.org/pypi/cylp/0.7.2

Note, to install these in zato, I had to use zato's pip to install numpy, scipy (as well as I had to use apt-get to install libblas-dev, libatlas-dev, and liblapack-dev so scipy could be compiled).

Then we write the solver interface, zato_tutorial/mips_simplex_solver.py and hot-deploy it.

Then we encode a TSP problem as a Linear Program for the simplex solver (https://en.wikipedia.org/wiki/Travelling_salesman_problem). Consider some $N\times N$ matrix $A$ representing travel costs between nodes( e.g., the cost of travel between node $i$ and node $j$ is $A(i,j)$. The goal is to find another $N\times N$ zero-one matrix of arcs between the nodes such that each node is visited exactly once and the minimum cost for the loop is found. In order to enforce the loop constraint, we need to introduce another variable $U$.

\begin{align}
&A \in \mathbb{R}^2 \\
&X \in \mathbb{Z}^2\\
&X = \textrm{argmin} (AX), \;\; \textrm{s.t.}\\
&\sum_{i\in N} {X_{ij}} = 1, \;\; \forall j\\
&\sum_{j\in N} {X_{ij}} = 1,\;\; \forall i\\
&X_{ij} \in \{0,1\}, \;\; \forall i,j\\
&U \in \mathbb{R}\\
& U_i - U_j + N*x_{ij} <= (N-1) , \;\; 1 \leq i \neq j \leq N \\
\end{align}

It should be noted that this formulation introduces $5N$ constraints and $N$ additional variables into an otherwise simple minimization.

**Also, as an aside,  there are (likely) bugs in Coin.**


In [23]:
import numpy as np
from cylp.cy.CyClpSimplex import CyClpSimplex
from cylp.py.modeling import CyLPModel, CyLPArray
#import warnings
#warnings.filterwarnings("ignore")

# Build a cost matrix
N = 5

np.random.seed(seed=0)
A = np.random.rand(N,N)
A += np.transpose(A)
np.fill_diagonal(A, 0)
A_col = CyLPArray(np.reshape(A, N**2))

#model = CyLPModel()
s = CyLPModel()
x = s.addVariable('x', N**2, isInt=True)
u = s.addVariable('u', N)

# Network Bounds
for i in range(N):

    
    # One edge from i
    #ix = np.array(range(N))+i*N
    ix = np.ravel_multi_index((i, range(N)), dims=(N, N))
    #print ix
    s += x[ix.tolist()].sum() == 1
    
    # One edge to i
    #ix = np.array(range(0, N**2, N)) + i
    ix = np.ravel_multi_index((range(N),i), dims=(N, N))
    #print ix
    s += x[ix.tolist()].sum() == 1
    
    s += 0 <= x <= 1 # MIP

# Don't allow routes to self  
s += CyLPArray(np.reshape(np.eye(N), N**2))*x == 0

# Force there to be a single cycle
np.ravel_multi_index
for i in range(N):
    for j in range(N):
        if i == j:
            continue
        ix = np.ravel_multi_index((i, j), dims=(N, N))
        #print ix, i, j
        s += u[i] - u[j] + N*x[[ix]] <= (N-1)

# Set the objective function
s.objective = A_col*x

s2 = CyClpSimplex(s)
s2.initialPrimalSolve()
print np.reshape(np.round(s2.primalVariableSolution['x']),(N,N))
#s2.primal()
#print np.reshape(np.round(s2.primalVariableSolution['x']),(N,N))

[[ 0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]]


In [24]:
print(dir(s2))

['CLP_addConstraint', 'CLP_addVariable', 'CLP_deleteConstraints', 'CLP_deleteVariables', 'CLP_dualConstraintSolution', 'CLP_getVarStatus', 'CLP_setVarStatus', 'Hessian', '__class__', '__delattr__', '__doc__', '__format__', '__getattribute__', '__hash__', '__iadd__', '__init__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_extractStartFinish', 'addConstraint', 'addConstraints', 'addVariable', 'addVariables', 'argWeightedMax', 'automaticScaling', 'basicVariables', 'basisInverse', 'checkVar', 'coefMatrix', 'constraintScale', 'constraints', 'constraintsLower', 'constraintsUpper', 'copyInIntegerInformation', 'cost', 'dual', 'dualConstraintSolution', 'dualTolerance', 'dualVariableSolution', 'dualWithPresolve', 'evaluateAt', 'extractCyLPModel', 'filterVars', 'flagged', 'freeOrSuperBasicVarInds', 'getACol', 'getBInvACol', 'getBInvCol', 'getBasisStatus', 'getCbcModel', 'getCoinInfinity', 'getComplementarity

In [25]:
cbc_model = s2.getCbcModel()
rsp = cbc_model.branchAndBound()
print cbc_model.isRelaxationOptimal()
print rsp
print s2.objectiveValue
print cbc_model.isRelaxationOptimal()

# Solve using primal Simplex
sol_x = cbc_model.primalVariableSolution['x']
print np.reshape(sol_x, (N,N))

True
solution
4.18512813236
True
[[ 0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.]]


In [26]:
%%file zato_tutorial/mips_simplex_solver.py

from zato.server.service import Service
import numpy as np
from cylp.cy.CyClpSimplex import CyClpSimplex
import tempfile

class GetClientAnswer(Service):
    def handle(self): # The only method that *has* to be defined. This is the event-triggered entry point.
        self.logger.info('Request MIPS Solution: {}'.format(self.request.payload))       
        response = {}
        q = self.request.payload['MIPS']
        N = self.request.payload['N']
        response['MIPS'] = q
        
        tmp_mps = tempfile.NamedTemporaryFile(suffix='.mps')
        tmp_mps.write(q)
        tmp_mps.seek(0)
            
        s3 = CyClpSimplex()
        s3.readMps(tmp_mps.name)
        tmp_mps.close()
        s3.initialPrimalSolve()
        s3.primal()
        
        #cbc_model = s3.getCbcModel()
        #rsp = cbc_model.branchAndBound()
        
        response['sol'] = np.reshape(np.round(s3.primalVariableSolution[:N**2]),(N,N))
        #response['sol'] = np.reshape(np.round(cbc_model.primalVariableSolution[:N**2]),(N,N))
        response['cost'] = s3.objectiveValue
        #response['optimal'] = cbc_model.isRelaxationOptimal()

        self.logger.info('Response: {}'.format(response))       
        
        # And return response to the caller
        self.response.payload = response



Overwriting zato_tutorial/mips_simplex_solver.py


In [27]:
import numpy as np
from cylp.cy.CyClpSimplex import CyClpSimplex
from cylp.py.modeling import CyLPModel, CyLPArray
import tempfile
import requests
import json
import time

#import warnings
#warnings.filterwarnings("ignore")

def buildTsp(N):
    # Build a cost matrix
    np.random.seed(seed=0)
    A = np.random.rand(N,N)
    A[np.triu_indices(N)] = 0
    A += np.transpose(A)
    np.fill_diagonal(A, 0)
    A_col = CyLPArray(np.reshape(A, N**2))

    #model = CyLPModel()
    s = CyLPModel()
    x = s.addVariable('x', N**2, isInt=True)
    u = s.addVariable('u', N, isInt=True)

    # Network Bounds
    for i in range(N):


        # One edge from i
        ix = np.array(range(N))+i*N
        #print ix
        s += x[ix.tolist()].sum() == 1

        # One edge to i
        ix = np.array(range(0, N**2, N)) + i
        #print ix
        s += x[ix.tolist()].sum() == 1

        s += 0 <= x <= 1 # MIP

    # Don't allow routes to self  
    s += CyLPArray(np.reshape(np.eye(N), N**2))*x == 0

    # Force there to be a single cycle
    np.ravel_multi_index
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            s += u[i] - u[j] + N*x[(i*N)+j] <= (N-1)


    # Set the objective function
    s.objective = A_col*x

    s2 = CyClpSimplex(s)
    tmp_mps = tempfile.NamedTemporaryFile(suffix='.mps')
    s2.writeMps(tmp_mps.name)
    tmp_mps.seek(0)
    return tmp_mps

def getAnswer(mps, N):
    payload = {'MIPS': ''.join(mps.read()), 'N': N}
    url = 'http://127.0.0.1:11223/solve/mips'
    #print payload
    r = requests.post(url, json=payload, timeout=60)
    return r.text

def probAndSolve(N):
    prob = buildTsp(n)
    sol_rsp = getAnswer(prob,n)
    #print sol_rsp
    sol = json.loads(sol_rsp)
    return sol
    
timeRange = range(3,25)
times = []
for n in timeRange:
    #%%timeit -n5 {probAndSolve(n)}
    tstart = time.clock()
    print n
    sol = probAndSolve(n)
    tend = time.clock()
    times.append(tend-tstart)
    if n < 8:
        print np.reshape(sol['sol'],(n,n))
    print sol['cost']

#print times


3
[[ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]]
1.8742433951
4
[[ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  0.]]
1.2177793262
5
[[ 0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]]
1.79123689916
6
[[ 0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]]
2.3971030481
7
[[ 0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.]]
2.05538190587
8
1.012482175
9
1.37236464227
10
1.23638193374
11
1.54910728856
12
1.70905181357
13
1.66732352782
14
2.28832644523
15
1.32043225261
16
1.70724739263
17
1.55025332406
18
1.23949528171
19
1.81966286631
20
1.18935748655
21
1.2993247367
22
1.62633778227
23
1.76076553868
24
1.17861980641


In [ ]:
import matplotlib.pyplot as plt
#print len(timeRange)
#print len(times)
#print times
plt.plot(timeRange, times)
plt.ylabel('Wall Time (s)')
plt.xlabel('Prob Size (N)')
plt.show()

###Potential issues with Zato
----
 - All servers must run on the same cluster, accessible via the file system. (Note, it does have the capacity to connect to OpenStack or Amazon's S3. Other solutions might involve using an ESB drive a set of ESBs.)
 - So far, I've not been able to interactively debug. That being said any error shows up in the logs, post-mortem, and in the response message.
 - Zato natively integrates with python and (the set of things that integrate with python). Though it provides an easy, modern way to add in other packages through other protocols.
                                                                             

-----
##Addendum: Checking for memory leaks in C-Types/PyObject with Valdgrind
Valgrind is an Open Source, Free C++ tool that can check for memory leaks, help debug code, and perform profiling. I personally use it to find problems that might not show up the unit tests or integration tests.This sections uses it to investigate one of the crufty PyObject examples I coded earlier. To follow along, you'll need the additional things, Valgrind, Python Source, and Python dependencies:

 - sudo apt-get install -y valgrind
 - sudo apt-get build-dep python2.7
 - apt-get source python2.7
 
Note, this last command downloads the source for Python in a local folder (in my case python2.7-2.7.6). This source needs to be compiled Python with debug flags and Valgrind (took about 2 minutes on a Core i7). Note, the below will put python in my /home/ashaver/valpython (which you probably don't want). First go read the information on python and Valgrind
 - http://svn.python.org/projects/python/trunk/Misc/README.valgrind
  - Ucomment Py_USING_MEMORY_DEBUGGER in Objects/obmalloc.c
  - Note, the top of that c-file has other things that we need to define in the configure command.
  - Uncomment the suppression in  Misc/valgrind-python.supp for PyObject_Free and PyObject_Realloc
 - build
  - ./configure --prefix=/home/ashaver/valpython --without-pymalloc --with-pydebug --with-valgrind
  - make -j4
  - make install

Now we have a python interpreter that works with Valgrind. So, we can go back and recompile an example, but with debugging and Valgrind. 

In [3]:
!/home/ashaver/valpython/bin/python2.7 --version
!valgrind --tool=memcheck --leak-check=yes /home/ashaver/valpython/bin/python2.7 --version

Python 2.7.6
==3971== Memcheck, a memory error detector
==3971== Copyright (C) 2002-2013, and GNU GPL'd, by Julian Seward et al.
==3971== Using Valgrind-3.10.0.SVN and LibVEX; rerun with -h for copyright info
==3971== Command: /home/ashaver/valpython/bin/python2.7 --version
==3971== 
Python 2.7.6
==3971== 
==3971== HEAP SUMMARY:
==3971==     in use at exit: 0 bytes in 0 blocks
==3971==   total heap usage: 0 allocs, 0 frees, 0 bytes allocated
==3971== 
==3971== All heap blocks were freed -- no leaks are possible
==3971== 
==3971== For counts of detected and suppressed errors, rerun with: -v
==3971== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


In [4]:
%%file code/hello_world.py
print "Hello World"

Writing code/hello_world.py


In [5]:
!valgrind /home/ashaver/valpython/bin/python2.7 code/hello_world.py

==3973== Memcheck, a memory error detector
==3973== Copyright (C) 2002-2013, and GNU GPL'd, by Julian Seward et al.
==3973== Using Valgrind-3.10.0.SVN and LibVEX; rerun with -h for copyright info
==3973== Command: /home/ashaver/valpython/bin/python2.7 code/hello_world.py
==3973== 
Hello World
[18300 refs]
==3973== 
==3973== HEAP SUMMARY:
==3973==     in use at exit: 953,884 bytes in 5,300 blocks
==3973==   total heap usage: 23,099 allocs, 17,799 frees, 4,604,692 bytes allocated
==3973== 
==3973== LEAK SUMMARY:
==3973==    definitely lost: 0 bytes in 0 blocks
==3973==    indirectly lost: 0 bytes in 0 blocks
==3973==      possibly lost: 291,902 bytes in 940 blocks
==3973==    still reachable: 661,982 bytes in 4,360 blocks
==3973==         suppressed: 0 bytes in 0 blocks
==3973== Rerun with --leak-check=full to see details of leaked memory
==3973== 
==3973== For counts of detected and suppressed errors, rerun with: -v
==3973== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)

In [15]:
%%file code/hello.cpp
#include <iostream>
#include <string.h>
using namespace std;
extern "C" {
    int main () {
        cout << "Hello Main2\n";
        return 42;
    }
    
    struct MsgWrapper {
        char * msgIn;
        char * msgOut;
    };
    
    void helloStrRefFromStruct(MsgWrapper s) {
        //strcpy(s.msgIn, "Aloha234\n");
        strcpy(s.msgIn, "Alohsadfgadfgaa\n");
    }  
     
    int missing_answer(int  i) {
        //non-leaky
        /*
        i = (42-i);
        return i;
        */
        
        //leaky
        ///*
        int * j = new int (42- i);
        return *j;
        //*/
    }
}

Overwriting code/hello.cpp


In [16]:
! rm bin/hello bin/libHello.so libHello.so
!g++ -c -g -std=c++11 -fPIC code/hello.cpp -o code/hello.o
!g++ -g -o bin/libHello.so -shared code/hello.o
!g++ -o bin/hello code/hello.o
!ln -s bin/libHello.so libHello.so
!valgrind --tool=memcheck --leak-check=yes bin/hello


==4031== Memcheck, a memory error detector
==4031== Copyright (C) 2002-2013, and GNU GPL'd, by Julian Seward et al.
==4031== Using Valgrind-3.10.0.SVN and LibVEX; rerun with -h for copyright info
==4031== Command: bin/hello
==4031== 
Hello Main2
==4031== 
==4031== HEAP SUMMARY:
==4031==     in use at exit: 0 bytes in 0 blocks
==4031==   total heap usage: 0 allocs, 0 frees, 0 bytes allocated
==4031== 
==4031== All heap blocks were freed -- no leaks are possible
==4031== 
==4031== For counts of detected and suppressed errors, rerun with: -v
==4031== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


In [2]:
%%file ./hello_wrapper.py
import ctypes
helloCpp = ctypes.CDLL('/home/ashaver/omaha_python_group_talks/07_15_2015_Integrating_Python_Adam_Shaver/libHello.so')
class MsgWrapper(ctypes.Structure):
    _fields_ = [("msgOut", ctypes.c_char_p), ("msgIn", ctypes.c_char_p)]
amsg = MsgWrapper("Goodbye", "Hello")
print(amsg.msgIn)
print(amsg.msgOut)
helloCpp.helloStrRefFromStruct(amsg)
print(amsg.msgIn)
print(amsg.msgOut)
print(helloCpp.missing_answer(24))

Overwriting ./hello_wrapper.py


In [3]:
!valgrind /home/ashaver/valpython/bin/python2.7 hello_wrapper.py

==4045== Memcheck, a memory error detector
==4045== Copyright (C) 2002-2013, and GNU GPL'd, by Julian Seward et al.
==4045== Using Valgrind-3.10.0.SVN and LibVEX; rerun with -h for copyright info
==4045== Command: /home/ashaver/valpython/bin/python2.7 hello_wrapper.py
==4045== 
Hello
Goodbye
==4045== Invalid write of size 8
==4045==    at 0x6E4B967: helloStrRefFromStruct (hello.cpp:17)
==4045==    by 0x6A340CB: ffi_call_unix64 (unix64.S:76)
==4045==    by 0x6A33AFB: ffi_call (ffi64.c:522)
==4045==    by 0x6A2291E: _call_function_pointer (callproc.c:836)
==4045==    by 0x6A23547: _ctypes_callproc (callproc.c:1183)
==4045==    by 0x6A1C045: PyCFuncPtr_call (_ctypes.c:3929)
==4045==    by 0x420D37: PyObject_Call (abstract.c:2529)
==4045==    by 0x4D4FE9: do_call (ceval.c:4239)
==4045==    by 0x4D439A: call_function (ceval.c:4044)
==4045==    by 0x4CF108: PyEval_EvalFrameEx (ceval.c:2666)
==4045==    by 0x4D1994: PyEval_EvalCodeEx (ceval.c:3253)
==4045==    by 0x4C7B5E: PyEval_EvalCode (ce

In [ ]:
!valgrind --leak-check=full  --suppressions=/home/ashaver/python2.7-2.7.6/Misc/valgrind-python.supp  /home/ashaver/valpython/bin/python2.7 hello_wrapper.py